In [16]:
import pandas as pd
import seaborn as sns

## Groupby Aggregations

Performing various aggregations in pandas. 

In [17]:
# Load in sample dataset. This dataset has a good variety of numeric / categorical columns. 
main_df = sns.load_dataset('taxis')
main_df.head(3)

,pickup,dropoff,passengers,distance,fare,tip,tolls,total,color,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough
0,2019-03-23 20:21:09,2019-03-23 20:27:24,1,1.60,7.0,2.15,0.0,12.95,yellow,credit card,Lenox Hill West,UN/Turtle Bay South,Manhattan,Manhattan
1,2019-03-04 16:11:55,2019-03-04 16:19:00,1,0.79,5.0,0.00,0.0,9.30,yellow,cash,Upper West Side South,Upper West Side South,Manhattan,Manhattan
2,2019-03-27 17:53:01,2019-03-27 18:00:25,1,1.37,7.5,2.36,0.0,14.16,yellow,credit card,Alphabet City,West Village,Manhattan,Manhattan


Suppose we wish to compute for each `pickup_borough` :
1. The median distance travelled
2. The max distance travelled
3. The total amount paid
4. The total number of trips

### Default `.agg()`

In [18]:
main_df.groupby('pickup_borough').agg({
    'distance' : ['median', 'max'] # For more than one aggregation per measure
    , 'total' : 'sum'
    , 'pickup' : 'count'
})

distance            total pickup
                 median    max       sum  count
pickup_borough                                 
Bronx               4.0  23.61   2253.76     99
Brooklyn            2.6  25.51   7367.48    383
Manhattan           1.5  28.30  87820.23   5268
Queens              6.1  36.70  20800.69    657

`.agg()` provides a quick way to compute aggregate values. However there are limitations. 
1. Multiple aggregations of the same dimension result in the multi column index that isn't as easily interpereted as a conventional column. Can be rectified by the custome renaming as mentioend later on. 
2. Cannot perform custom aggregations. 

### `pd.NamedAgg()`

Calling `.agg()` but this time passing `NamedAgg()` arguments. These allow us to specify : 

- What the resulting aggregated columns are named
- Custom aggregation functions should we choose

__NOTE__ when creating custom aggregation functions, the input will be the pandas Series version of the column specified. 

[Reference Article](https://deanla.com/pandas_named_agg.html)

In [19]:
tdf = main_df.groupby('pickup_borough').agg(
    median_distance = pd.NamedAgg(column='distance', aggfunc='median')
    , max_distance = pd.NamedAgg(column='distance', aggfunc=lambda x : x.max()) # Creating custom agg function
    , total_paid = pd.NamedAgg(column='total', aggfunc='sum')
    , n_trips = pd.NamedAgg(column='total', aggfunc='size')
)
tdf

,median_distance,max_distance,total_paid,n_trips
pickup_borough,,,,
Bronx,4.0,23.61,2253.76,99
Brooklyn,2.6,25.51,7367.48,383
Manhattan,1.5,28.30,87820.23,5268
Queens,6.1,36.70,20800.69,657


The downside to this approach is that it's more verbose and isn't as quick to write. 

## Stacking & Unstacking

Exploration of pivoting / unpivoting using the inbuilt `.stack()` & `.unstack()` functionality. 

[Reference Video](https://www.youtube.com/watch?v=kJsiiPK5sxs)

In [20]:
# Load in sample dataset. This dataset has a good variety of numeric / categorical columns. 
main_df = sns.load_dataset('taxis')
main_df.head(3)

,pickup,dropoff,passengers,distance,fare,tip,tolls,total,color,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough
0,2019-03-23 20:21:09,2019-03-23 20:27:24,1,1.60,7.0,2.15,0.0,12.95,yellow,credit card,Lenox Hill West,UN/Turtle Bay South,Manhattan,Manhattan
1,2019-03-04 16:11:55,2019-03-04 16:19:00,1,0.79,5.0,0.00,0.0,9.30,yellow,cash,Upper West Side South,Upper West Side South,Manhattan,Manhattan
2,2019-03-27 17:53:01,2019-03-27 18:00:25,1,1.37,7.5,2.36,0.0,14.16,yellow,credit card,Alphabet City,West Village,Manhattan,Manhattan


Observe that the dataset is in a [tidy format](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html#:~:text=Tidy%20data%20is%20a%20standard,with%20observations%2C%20variables%20and%20types.), where : 

1. Every column is a variable
2. Every row is an observation
3. Every cell is a single value

Data in this format is easily understood.

### Unstacking (or Pivoting) / Pivoting Long to Wide

It is typical to want to aggreagate data by one or more dimensions. Suppose we wanted __the breakdown of the *number of* rides with each payment type by pickup borough__

In [21]:
# Group by the two items
df = main_df.groupby(['payment', 'pickup_borough'])\
    .agg({'pickup' : 'size'})\
    .rename(columns={'pickup' : 'n'})
df

n
payment     pickup_borough      
cash        Bronx             25
            Brooklyn         119
            Manhattan       1397
            Queens           266
credit card Bronx             74
            Brooklyn         261
            Manhattan       3839
            Queens           383

This gives us the multi indexed aggregation. However this view isn't great for making quick comparisons across dimensions (eg. how do cash & credit card payments differ for the Bronx?)

Converting the `pickup_borough` to separate columns, or __pivoting from long to wide__ would solve this problem.

`df.unstack(level=-1)` lets us to this. In short, we turn a specified `level` value (one of the index columns) from an indexing row to a series of columns. 

By default `level=-1` refers to the outermost column index. Note that the __innermost index is to the left, while the outermost index is on the right__

In [22]:
# The boroughs were the second level index, on the right of payment methods
df.unstack(level=-1)

n                          
pickup_borough Bronx Brooklyn Manhattan Queens
payment                                       
cash              25      119      1397    266
credit card       74      261      3839    383

In [23]:
# Here we choose to pivot on the innermost index of payment method instead, so we need to specify the level=0
df = df.unstack(0)
df

n            
payment         cash credit card
pickup_borough                  
Bronx             25          74
Brooklyn         119         261
Manhattan       1397        3839
Queens           266         383

The comparison is now plain to see. However there arises a new issue with the column indexing. 

The columns are now a MultiIndex, with two layers to the columns. 

In [24]:
df.columns

MultiIndex([('n',        'cash'),
            ('n', 'credit card')],
           names=[None, 'payment'])

It is possible to index the data using [tuple indexing](https://pandas.pydata.org/docs/user_guide/advanced.html), however most of the time it easier to simply concat the two layers together. 

In [25]:
df.columns =  ['__'.join(col).strip() for col in df.columns.values]
df

,n__cash,n__credit card
pickup_borough,,
Bronx,25,74
Brooklyn,119,261
Manhattan,1397,3839
Queens,266,383


This manual reset also removes the two layer column names. 

In [26]:
df.columns

Index(['n__cash', 'n__credit card'], dtype='object')

### Stacking (or unpivoting) / Pivoting Wide to Long

This function does the opposite of `unstack()`. We take the pivoted dataframe and convert it __back to its tidy format__.

In [34]:
df.unstack()

                pickup_borough
n__cash         Bronx               25
                Brooklyn           119
                Manhattan         1397
                Queens             266
n__credit card  Bronx               74
                Brooklyn           261
                Manhattan         3839
                Queens             383
dtype: int64

Observe that this causes a problem: the object returned is a series by default. If we want to convert it back to the tidy format we can reset the index.

We will also need to tidy up some of the column names, as well as some column values (for the column that was unpivoted)

In [35]:
df = df.stack().reset_index()\
    .rename(columns = {'level_1' : "payment_method", 0:'n'})

df['payment_method'] = df['payment_method'].str.removeprefix("n__")
df

,pickup_borough,payment_method,n
0,Bronx,cash,25
1,Bronx,credit card,74
2,Brooklyn,cash,119
3,Brooklyn,credit card,261
4,Manhattan,cash,1397
5,Manhattan,credit card,3839
6,Queens,cash,266
7,Queens,credit card,383


Now the dataframe is back to it's original tidy format, although the multi index no longer exists. 

If you wish you may manually set it yourself. 

In [38]:
df.set_index(['payment_method', 'pickup_borough'])

,,n
payment_method,pickup_borough,
cash,Bronx,25
credit card,Bronx,74
cash,Brooklyn,119
credit card,Brooklyn,261
cash,Manhattan,1397
credit card,Manhattan,3839
cash,Queens,266
credit card,Queens,383


Now we are back to the multi-indexed version as a dataframe.